In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install PyPDF2

In [1]:
from PyPDF2 import PdfReader

In [2]:
text = ""
pdf_reader = PdfReader('/kaggle/input/tsechack/Vikas_Resume.pdf')
for page in pdf_reader.pages:
    text += page.extract_text()


In [3]:
# print(text)

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": None,
    "do_sample": False,
}

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Starting the Interview

In [22]:
list_of_quesn = []

In [23]:
start_question = [
{"role": "system", "content": 
"You are a helpful AI assistant."},
    
{"role": "user", "content": 
f"""
{text}

You have candidate resume above ask one short technical question which gives good start to interview process.
Format of the output has to be just one question no explaination needed.
""".strip()},
    
]

output = pipe(start_question, **generation_args)
res = output[0]['generated_text'].strip()

list_of_quesn.append(res)

print(res)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


What machine learning model did you fine-tune for the forest fire management system, and what was the impact on development time?


In [31]:
import io
from google.oauth2 import service_account
from google.cloud import speech

client_file = '/kaggle/input/google-audio/audio.json'
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)

audio_file = 'interview1.wav'
with io.open(audio_file, 'rb') as f:
    content = f.read()
    audio = speech.RecognitionAudio(content=content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US'
)

answer = client.recognize(config=config, audio=audio)
print(answer)

## Continue the Interview

In [24]:
continue_question = [
{"role": "system", "content": 
"You are a helpful AI assistant."},
    
{"role": "user", "content": 
f"""
{res}

Candidate response:
{answer}

You have to ask a short continue question based on candidate response.
Format of output would be just one new question.
""".strip()},
    
]

output = pipe(continue_question, **generation_args)
con_res = output[0]['generated_text'].strip()

list_of_quesn.append(con_res)

print(con_res)

How did the implementation of Q-LoRA and 4-bit quantization specifically contribute to the accuracy of the model in the context of forest fire management?


In [25]:
new_question = [
{"role": "system", "content": 
"You are a helpful AI assistant."},
    
{"role": "user", "content": 
f"""
List of questions:
{list_of_quesn}

You have to ask interview a new question from the below resume which is not from the above list.

{text}
Format of output would be just one new question.
""".strip()},
    
]

output = pipe(new_question, **generation_args)
new_res = output[0]['generated_text'].strip()

list_of_quesn.append(new_res)

print(new_res)

How did the integration of Google PaLM and LangChain in the Ettara Cafe project enhance the efficiency of order placements and customer feedback processing, and what were the key challenges faced during this implementation?


In [27]:
list_of_quesn

['What machine learning model did you fine-tune for the forest fire management system, and what was the impact on development time?',
 'How did the implementation of Q-LoRA and 4-bit quantization specifically contribute to the accuracy of the model in the context of forest fire management?',
 'How did the integration of Google PaLM and LangChain in the Ettara Cafe project enhance the efficiency of order placements and customer feedback processing, and what were the key challenges faced during this implementation?']

## Assessment of Interview

In [39]:
assessment = [
{"role": "system", "content": 
"You are a helpful AI assistant."},
    
{"role": "user", "content": 
f"""
Question:
{res}

Candidate Response
{answer}

You have to assess candidate response and rate them from 5 in terms of grammar, answer relevancy, and fluency.
Format the answer in list [grammar_score, relevancy_score, fluency] and just return list no explanation needed.
""".strip()},
    
]

output = pipe(assessment, **generation_args)
assess = output[0]['generated_text'].strip()

print(assess)

[grammar_score: 3, relevancy_score: 4, fluency: 3]


## Visual Tracking

In [45]:
!pip install mediapipe

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 44.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is inc

In [12]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

cap = cv2.VideoCapture("/kaggle/input/interview/inter1.mp4")

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            break
            
        # If loading a video, use 'break' instead of 'continue'.
#         continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
            
        # Flip the image horizontally for a selfie-view display.
#         cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))

        if cv2.waitKey(5) & 0xFF == 27:
            break
    
    cap.release()

W0000 00:00:1727701143.689767     235 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727701143.715662     234 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/conda/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1367: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'


ERROR: Could not find a version that satisfies the requirement google-colab (from versions: none)
ERROR: No matching distribution found for google-colab
